In [2]:
# packages
import requests
from bs4 import BeautifulSoup
import re


In [4]:
KEYWORD = 'zoom'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    
url = 'https://www.webull.com/newslist/nasdaq-zm'

# send a GET request to the URL and parse the HTML content with Beautiful Soup
soup = BeautifulSoup(requests.get(url, headers=headers).content, 'html.parser')

In [29]:
news_list = soup.find('div', class_='newsListContent')
# print(news_list)

articles = news_list.find_all('div', class_='ssr414261') # headlines
# print(articles)

taglines = news_list.find_all('div', class_='ssr414262') 
# print(taglines)

headers = []
dates = []

for article in articles:
    header = article.text
    headers.append(header)

for tag in taglines:
    date = tag.text.split('·')[1].strip()
    dates.append(date)

In [31]:
print(headers)

['Guru Fundamental Report for ZM - Benjamin Graham', 'ARKK, ZM, ROKU, COIN: Large Outflows Detected at ETF', '5 Traits of Legendary Investors', 'Oracle And Zoom Join Forces To Expand Access To Virtual Care', 'Apple, Zoom Video And 2 Other Stocks Insiders Are Selling', 'Zoom Forays into Employee Engagement Space: ETFs in Focus', 'Zoom Stock Slides as Analyst Warns of Weakening Demand', 'After Hours Most Active for Apr 17, 2023 :  SWN, ZM, BAC, QQQ, AAPL, GOOGL, PFE, MSFT, NRG, AMC, AMZN, FE', 'Noteworthy Monday Option Activity: STNG, ZM, BLNK', "Zoom's Pain Will Continue To Be Microsoft's Gain", 'Zoom stock slips as Citi opens up downside catalyst watch', 'Analysts’ Opinions Are Mixed on These Technology Stocks: My Size (MYSZ) and Zoom Video Communications (ZM)', 'Zoom Video to acquire human-resources platform Workvivo for undisclosed sum', 'BRIEF-Zoom To Acquire Workvivo To Bolster The Employee Experience Offering', 'Insider Sell: Zoom Video Communications', 'BRIEF-Zoom Says To Acquire

In [32]:
print(dates)

['8h ago', '11h ago', '13h ago', '13h ago', '13h ago', '16h ago', '1d ago', '1d ago', '1d ago', '1d ago', '1d ago', '1d ago', '4d ago', '4d ago', '4d ago', '4d ago', '4d ago', '4d ago', '4d ago', '4d ago']


In [33]:
import pandas as pd

# Combine the two lists into a DataFrame
df = pd.DataFrame({'date': dates, 'news_headline': headers})

# Save the DataFrame as a CSV file
df.to_csv("testing_webull.csv", index=False)

# dynamically scrape

In [47]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# set up the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# navigate to the webpage
driver.get("https://www.webull.com/newslist/nasdaq-zm")

# wait for the "More" button to appear and click it repeatedly until the terminating phrase is found
start_phrase = "Rosenblatt Initiates Coverage On Zoom Video Comms with Neutral Rating, Announces Price Target of $75"
end_phrase = "How Zoom's Mission Could Help It Lower Its Biggest Expense"
scraped_headers = []
scraped_dates = []

continue_scrape = False

while True:
    try:
        more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                                (By.XPATH, "//div[@class='csr19'][text()='More']")))
        more_button.click()
        time.sleep(1)

        headers = driver.find_elements_by_class_name("csr14")
        dates = driver.find_elements_by_class_name("csr15")

        for i in range(len(headers)):
            header = headers[i]
            date = dates[i]

            if header.text == start_phrase:
                continue_scrape = True
                scraped_headers.append(header.text)
                scraped_dates.append(date.text.split('·')[1].strip())

            if header.text == end_phrase:
                driver.quit()
                print(scraped_headers[1:])
                print(scraped_dates[1:])
                break

            if continue_scrape:
                scraped_headers.append(header.text)
                scraped_dates.append(date.text.split('·')[1].strip())
    except:
        driver.quit()
        # print(scraped_headers)
        break


/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_8020/1616448084.py:30: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("csr14")
/var/folders/cf/28wm9krx5816hxprmj3lstxh0000gn/T/ipykernel_8020/1616448084.py:31: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  dates = driver.find_elements_by_class_name("csr15")


['Rosenblatt Initiates Coverage On Zoom Video Comms with Neutral Rating, Announces Price Target of $75', 'Insider Sell: Zoom Video Communications', 'ARK Innovation ETF Experiences Big Inflow', "Zoom Beat Earnings Expectations, But Here's Why I Still Wouldn't Buy the Stock", "Microsoft's ChatGPT Ended Zoom Video's Growth Ambitions", 'Zoom Video Communications Still Offers Strong Potential Sitting Just Above Its 52-Week Low', 'Notable Friday Option Activity: ZM, MCD, FLS', 'Zoom Video (ZM) Starts Partnership With Major League Baseball', 'Better Buy: Zoom vs. Okta', 'Why This RingCentral Analyst Is Turning Bearish']
['03/24 04:50', '03/23 13:05', '03/23 10:51', '03/23 07:37', '03/21 14:59', '03/21 11:00', '03/17 15:26', '03/17 10:53', '03/17 10:23', '03/17 10:22']
